In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/ansh.arora/IIITA_Winter21/YOLOv5-CustomDataset/yolov5


In [2]:
!pip install -r requirements.txt

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d awsaf49/bdd100k-dataset --force
# !kaggle datasets download -d solesensei/solesensei_bdd100k
!kaggle datasets download -d nguyentrongquocdat/object-detection-for-autonomous-vehicles

In [ ]:
%cd /home/ansh.arora/IIITA_Winter21/YOLOv5-CustomDataset/

In [ ]:
!ls

In [ ]:
!mv Object\ Detection Object_Detection

In [ ]:
!ls

In [ ]:
# !unzip YOLOv5-CustomDataset/object-detection-for-autonomous-vehicles.zip

In [ ]:
# !rm /content/bdd100k-dataset.zip

In [ ]:
%cd /content/Object Detection/image/train
!ls | wc -l
%cd /content

In [8]:
# import torch
# from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
# import xml.etree.ElementTree as ET
# from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [ ]:
import json
# Function to get the data from JSON Annotation
def extract_info_from_json(json_file):
    json_file = open(json_file)
    data = json.load(json_file)
    
    image_size = [1280,720,3]
    
    annotations = []
    # Parse the JSON Tree
    print(len(data))
  
    for idx in range(len(data)):
      # Initialise the info dict 
      info_dict = {}
      info_dict['bboxes'] = []
      
      info_dict['filename'] = data[idx]['name']
      info_dict['image_size'] = tuple(image_size)
  
      # if(data[idx]['labels']==None):
      #   annotations.append(info_dict)
      #   continue
      try:
        for label in data[idx]['labels']:
          bbox={}
          bbox["class"] = label["category"]

          for coord in label['box2d']:
            value = label['box2d'][coord]
            if(coord == 'x1'):
              coord = 'xmin'
            elif(coord == 'x2'):
              coord ='xmax'
            elif(coord == 'y1'):
              coord = 'ymin'
            elif(coord == 'y2'):
              coord = 'ymax'
            bbox[coord] = int(value)

          info_dict['bboxes'].append(bbox)
      except:
        annotations.append(info_dict)
        print(data[idx])
        
      annotations.append(info_dict)
    return annotations
    
    

In [ ]:
# print(extract_info_from_json('/content/bdd100k/labels/box_track_20/train/0000f77c-62c2a288.json'))
# print(extract_info_from_json('/content/Object Detection/label/train.json'))

In [ ]:
!mkdir 'Object_Detection/label/train' 
!mkdir 'Object_Detection/label/val' 
!mkdir 'Object_Detection/label/test'
!mkdir 'Object_Detection/image/val'
!mkdir 'Object_Detection/image/train_new'
!mkdir 'Object_Detection/label/train_new' 

In [ ]:
#  !mkdir /content/bdd100k/labels/train /content/bdd100k/labels/val /content/bdd100k/labels/test

In [ ]:
# from google.colab import drive
# drive._mount('/content/drive')

In [ ]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"pedestrian":0,
                            "rider":1,
                            "car":2,
                            "truck":3,
                            "bus":4,
                            "train":5,
                            "motorcycle":6,
                            "bicycle":7,
                            "traffic light":8,
                            "traffic sign":9,
                            "trailer":10,
                            "other person": 11,
                            "other vehicle":12}

# class_name_to_id_mapping = {"pedestrian":4,
#                             "rider":7,
#                             "car":3,
#                             "truck":6,
#                             "bus":5,
#                             "train":10,
#                             "motorcycle":9,
#                             "bicycle":8,
#                             "traffic light":1,
#                             "traffic sign":2,
#                             "trailer":13,
#                             "other person": 12,
#                             "other vehicle":11}


# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict,type):
    print_buffer = []
    # For each bounding box
    # if(int(len(info_dict["bboxes"])==0)):
    #   save_file_name = os.path.join("/content/bdd100k/bdd100k/labels/"+type, info_dict["filename"].replace("jpg", "txt"))
    #   print("", file= open(save_file_name, "w"))
    for b in info_dict["bboxes"]:
        try:
          class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print(b)
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("Object_Detection/label/"+type, info_dict["filename"].replace("jpg", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# # Get the annotations
# # annotations = [os.path.join('/content/bdd100k/labels/box_track_20/train', x) for x in os.listdir('/content/bdd100k/labels/box_track_20/train') if x[-4:] == "json"]
# annotations.sort()
# print(annotations)


# # Convert and save the annotations
# for ann in tqdm(annotations):
#     info_dict = extract_info_from_json(ann)
#     for i in info_dict:
#       # print(i)
#       convert_to_yolov5(i,"train")
#       # print("--------------------")      
# annotations = [os.path.join('/content/bdd100k/labels/box_track_20/train', x) for x in os.listdir('/content/bdd100k/labels/box_track_20/train') if x[-3:] == "txt"]
# print(annotations)

In [ ]:
# Assuming you're in the data folder
# !cat /content/bdd100k/bdd100k/labels/det_v2_train_release.json

In [ ]:
#train annotations
info_dict = extract_info_from_json('Object_Detection/label/train.json')
# print(info_dict)
for i in info_dict:
    convert_to_yolov5(i,"train")

In [ ]:
#train annotations
info_dict = extract_info_from_json('Object_Detection/label/test.json')
# print(info_dict)
for i in info_dict:
    convert_to_yolov5(i,"test")

In [ ]:
# annotations = [os.path.join('/content/bdd100k/bdd100k/labels/train', x) for x in os.listdir('/content/bdd100k/bdd100k/labels/train') if x[-3:] == "txt"]
# print(annotations)

In [ ]:
# !rm -rf /content/Object Detection/label/val

In [ ]:
# %cd /content/Object Detection/label/
# !rm -rf val

In [ ]:
%cd /content/bdd100k/bdd100k/images/100k/train
!ls | wc -l
%cd /content/bdd100k/bdd100k/labels/train
!ls | wc -l

In [ ]:
# !unzip /content/drive/MyDrive/bdd100k_det_20_labels_trainval.zip -d /content

In [ ]:
# #train annotations
# info_dict = extract_info_from_json('/content/bdd100k/bdd100k/labels/det_v2_val_release.json')
# # print(info_dict)
# for i in info_dict:
#     convert_to_yolov5(i,"val")

In [ ]:
%cd /content/Object Detection/label/val
!ls | wc -l
%cd /content/bdd100k/bdd100k/labels/val
!ls | wc -l

In [1]:
import os

In [2]:
annotations = [os.path.join('Object_Detection/label/train', x) for x in os.listdir('Object_Detection/label/train') if x[-3:] == "txt"]

In [3]:
images = [os.path.join('Object_Detection/image/train', x) for x in os.listdir('Object_Detection/image/train')]

In [4]:
# import required module
# import os
# assign directory
directory = 'Object_Detection/image/train'
labels = 'Object_Detection/label/train/'
# iterate over files in
# that directory
ans=0
for filename in os.listdir(directory):
  if(os.path.exists(labels+filename.replace("jpg", "txt"))==False):
    print(filename)
    os.remove(directory+'/'+filename)
    ans+=1
print(ans)
  # break
# for filename in os.listdir(directory):
#   print(filename)
# 	f = os.path.join(directory, filename)
# 	# checking if it is a file
# 	# if os.path.isfile(f):
#   #    print(filename)  
#   break


0


In [ ]:
%cd Object_Detection/image/train
!ls | wc -l

In [5]:
%cd /home/ansh.arora/IIITA_Winter21/YOLOv5-CustomDataset

/home/ansh.arora/IIITA_Winter21/YOLOv5-CustomDataset


In [9]:
# random.seed(0)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    print(transformed_annotations)

    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)),outline ="red")
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
# annotation_file = "/content/bdd100k/labels/box_track_20/train/0000f77c-62c2a288-0000020.txt"
# print(annotation_file.index("0000f77c-62c2a288-0000020"))
# print(annotation_file.index("-0000020"))
# print(annotation_file.index(".txt"))
file_name = annotation_file[29:59]
print(annotation_file)
print(file_name)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
    

#Get the corresponding image file
image_file = annotation_file.replace("label/", "image/").replace("txt", "jpg")
print(image_file)
# image_file = "/content/bdd100k/images/track/train/0000f77c-62c2a288/0000f77c-62c2a288-0000020.jpg"
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

NameError: name 'class_name_to_id_mapping' is not defined

In [11]:
images.sort()
annotations.sort()
# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)

In [12]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'Object_Detection/image/train_new')
move_files_to_folder(val_images, 'Object_Detection/image/val')
# move_files_to_folder(test_images, 'images/test/')
# move_files_to_folder(train_annotations, 'annotations/train/')
# move_files_to_folder(val_annotations, 'annotations/val/')
# move_files_to_folder(test_annotations, 'annotations/test/')
move_files_to_folder(train_annotations, 'Object_Detection/label/train_new')
move_files_to_folder(val_annotations, 'Object_Detection/label/val')

In [13]:
# !mv /content/Object_Detection/images/train_new /content/Object_Detection/images/train
# !mv /content/Object_Detection/labels/train_new /content/Object_Detection/labels/train
!mv Object_Detection/image Object_Detection/images
!mv Object_Detection/label Object_Detection/labels

In [ ]:
from google.colab import drive
drive._mount('/content/gdrive')


In [ ]:
!zip -r '/content/gdrive/MyDrive/IIITA_Winter/bdd100k.zip' /content/Object_Detection

In [ ]:
# !unzip /content/gdrive/MyDrive/IIITA_Winter/bdd100k.zip